Recently,a movie called "alien covenant" caught a lot of people's eye. Just as a reviewer puts it-sci-fi meets classic literature in 'Alien: Covenant', this sci-fi film mixed itself with multiple classical literature elements to distinguish itself from other sci-fi films. 

These literature elements brought the film some aroma of exotic spices and impressed me a lot. And the one intrigues me most comes from the scence where the robot David cited a Shelley's poem --"Look on my works, ye Mighty, and despair". Though , David didn't mention that he want to become the king of the king, what he did was self-evident. 

While David praises Shelley's poem as remarkable, I am somewhat puzzled. If David was so creative,at least he himself thought so, Shouldn't he produce an original masterpiece by himself rather than recite Shelley's.

Well, this is something deep learning can exzactly do, without further ado, let's make an AI to do just that-- create a new Shelley's style poem. Even the AI I created is not as powerfull as David,at least, it creates a poem better than he can do.

Here is the plan I come about

First, I need to get all the poems written by Shelley.

Then, I'll use the RNN Neural Network with LSTM to train the data

Lastly, with saved parameters, I can create as many poems as possible


First, I crawled all Shelley's peoms from a website. The website contains 327 poems written by Shelley and the crawling process went smoothly. In a few minutes , all poems were downloaded. 

The next thing to do is to use a RNN architecture to train the AI. the training process also went smoothly.

Here is a piece of poem written by AI, see, it wasn't that bad after all, at least, it surpasses mine. 


then, 
fraud then thou, 
but more dear to life, 
as from the spirit's wilderness, 
the thick which thou withered, 
for the song, 
when the moon is awakened, 
the flood is buried, 
but gentle gentle was slumber thus in her grave. 
there is a reward of clouds, and the swallow the water fell. 
the coursers springs of the soul it was too. 
then shrinks back, and i cry about to know that others fall, 
knows by looks, 
we moulded in mountains. 
but like shadow after friends and filled with thine own sway;-- from the earth’s waves are shaken, 
the nations & sorrows not the speed of bacchus, 
in the blood of the ocean cling, 
with living green and black both. though far bringing, and when i feeble, i think i arose the night the eyes, like wrecks, 
in the surge of heaven, where high once lighted green the red shades down in clear up in the place of the unfathomable ocean, 
night in the eden of one fair dew, 
two by pain


Here is how I created the ID iteration webcrawler to fetch all the tittle and its hyperlinks, after that, for  each crawled link, I'll catch the linked poem and download it into the text file. 

In [2]:
import urllib.request
import re
from urllib.error import URLError, HTTPError, ContentTooShortError
from urllib.parse import urljoin
from urllib.parse import urlparse
import time


In [3]:
def download(url, user_agent='wswp', num_retries=4, charset='utf-8', proxy=None):
    """ Download a given URL and return the page content
        args:
            url (str): URL
        kwargs:
            user_agent (str): user agent (default: wswp)
            charset (str): charset if website does not include one in headers
            proxy (str): proxy url, ex 'http://IP' (default: None)
            num_retries (int): number of retries if a 5xx error is seen (default: 2)
    """
    print('Downloading:', url)
    request = urllib.request.Request(url)
    request.add_header('User-agent', user_agent)
    try:
        if proxy:
            proxy_support = urllib.request.ProxyHandler({'http': proxy})
            opener = urllib.request.build_opener(proxy_support)
            urllib.request.install_opener(opener)
        resp = urllib.request.urlopen(request)
        cs = resp.headers.get_content_charset()
        if not cs:
            cs = charset
        html = resp.read().decode(cs)
    except (URLError, HTTPError, ContentTooShortError) as e:
        print('Download error:', e.reason)
        html = None
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                # recursively retry 5xx HTTP errors
                return download(url, num_retries - 1)
    return html

import itertools 
from bs4 import BeautifulSoup as bs

def crawl_site(url):

    titles=[]
    linked_address = []
    for page in itertools.count(1): 
        if page > 9:
            break
        pg_url = '{}{}{}'.format(url, page,'/?a=a&l=3&y=') 
        html = download(pg_url) 
        if html is None:
            break 
        soup = bs(html,'html5lib')
        lik = soup.find_all('td',attrs={'class':'title'})
        
        table = soup.find('table',attrs ={'class':'poems'})
        h_links = table.find_all('a',href=True)
        tittle = [(t.text) for t in lik]
        
        r_link = [t.get('href') for t in h_links]
        linked_address.append(r_link)
        titles.append(tittle)


    return titles,linked_address


In [5]:
all_tittles,linked_address = crawl_site('https://www.poemhunter.com/percy-bysshe-shelley/poems/page-')

Downloading: https://www.poemhunter.com/percy-bysshe-shelley/poems/page-1/?a=a&l=3&y=
Downloading: https://www.poemhunter.com/percy-bysshe-shelley/poems/page-2/?a=a&l=3&y=
Downloading: https://www.poemhunter.com/percy-bysshe-shelley/poems/page-3/?a=a&l=3&y=
Downloading: https://www.poemhunter.com/percy-bysshe-shelley/poems/page-4/?a=a&l=3&y=
Downloading: https://www.poemhunter.com/percy-bysshe-shelley/poems/page-5/?a=a&l=3&y=
Downloading: https://www.poemhunter.com/percy-bysshe-shelley/poems/page-6/?a=a&l=3&y=
Downloading: https://www.poemhunter.com/percy-bysshe-shelley/poems/page-7/?a=a&l=3&y=
Downloading: https://www.poemhunter.com/percy-bysshe-shelley/poems/page-8/?a=a&l=3&y=
Downloading: https://www.poemhunter.com/percy-bysshe-shelley/poems/page-9/?a=a&l=3&y=


In [6]:

t = [x for tit in all_tittles for x in tit]
h = [x for tit in linked_address for x in tit]
t = [x[11:-10] for x in t]


In [7]:
def abs_links(start_url,all_links):
    abslinks=[]
    for link in all_links:
        abs_link = start_url+link
        abslinks.append(abs_link)
    return abslinks

In [8]:
starturl = 'https://www.poemhunter.com'

abslinks = abs_links(starturl,h)

In [9]:
def get_poems(urls):
    poems=[]
    for url in urls: 
        html = download(url) 
        print("{}".format(urls.index(url)/len(urls)),"are completed")
        if html is None: 
            break 
        soup = bs(html,'html5lib')
        lik = soup.find('div',attrs={'class':'KonaBody'})
        poem = lik.find('p').text
        poems.append(poem)
    return poems

In [10]:
all_poem = get_poems(abslinks)

Downloading: https://www.poemhunter.com/poem/the-retrospect-cwm-elan-1812/
Downloading: https://www.poemhunter.com/poem/to-sophia-miss-stacey/
Downloading: https://www.poemhunter.com/poem/to-the-lord-chancellor/
Downloading: https://www.poemhunter.com/poem/the-sepulchre-of-memory/
Downloading: https://www.poemhunter.com/poem/to-edward-williams/
Downloading: https://www.poemhunter.com/poem/to-harriet-it-is-not-blasphemy-to-hope-that-heaven/
Downloading: https://www.poemhunter.com/poem/from-laon-and-cythna-or-the-revolution-of-the-golden-city/
Downloading: https://www.poemhunter.com/poem/to-harriet/
Downloading: https://www.poemhunter.com/poem/dark-spirit-of-the-desart-rude/
Downloading: https://www.poemhunter.com/poem/the-wandering-jew-s-soliloquy/
Downloading: https://www.poemhunter.com/poem/to-mary-16/
Downloading: https://www.poemhunter.com/poem/the-tower-of-famine/
Downloading: https://www.poemhunter.com/poem/the-zucca/
Downloading: https://www.poemhunter.com/poem/the-rude-wind-is-s

In [11]:
from string import punctuation

poems = [''.join(poem.split('\n')) for poem in all_poem]



In [12]:
all_text = ''
for i in poems:
    all_text += i

In [13]:
txet_file = open('data/poem.txt','w')
txet_file.write(all_text)
txet_file.close()

In [15]:
import csv
with open('data/peom.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(['title','content'])
    for i in range(len(t)):
        spamwriter.writerow([t[i],poems[i]])